# 12_继承的优缺点

## 12.1 子类化内置类型很麻烦

- 直接子类化内置类型(dict, list, str) 容易出错，内置方法通常会忽略用户覆盖的方法。
- 用户自定义的类应继承 collections 模块中的 `UserDict, UserList, UserString`,易于拓展。
- 内置类型为C语言实现的，容易出现问题。collections是由Python编写的类。

#### 1 内置类型 dict 的 `__init__` 和 `__update__` 方法会忽略覆盖的 `__setitem__`

In [2]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)

d = DoppelDict(one=1)  # 并没有生效
print(f"d \t\t {d}")

d.update(three=3)  # 并没有生效
print(f"d \t\t {d}")

d['two'] = 2
print(f"d \t\t {d}")

d 		 {'one': 1}
d 		 {'one': 1, 'three': 3}
d 		 {'one': 1, 'three': 3, 'two': [2, 2]}


In [9]:
import collections

class DoppelDict(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)

d = DoppelDict(one=1)  # 并没有生效
print(f"d \t\t {d}")

d.update(three=3)  # 并没有生效
print(f"d \t\t {d}")

d['two'] = 2
print(f"d \t\t {d}")

d 		 {'one': [1, 1]}
d 		 {'one': [1, 1], 'three': [3, 3]}
d 		 {'one': [1, 1], 'three': [3, 3], 'two': [2, 2]}


#### 2 `dict.update` 方法会忽略 `AnswerDict.__getitem__`方法

In [8]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42
    
ad = AnswerDict(a='foo')
print(ad['a'])

d = {}
d.update(ad)
print(d['a'])
print(d)

42
foo
{'a': 'foo'}


In [10]:
import collections

class AnswerDict(collections.UserDict):
    def __getitem__(self, key):
        return 42
    
ad = AnswerDict(a='foo')
print(ad['a'])

d = {}
d.update(ad)
print(d['a'])
print(d)

42
42
{'a': 42}


## 12.2 多重继承

In [14]:
class A:
    def ping(self):
        print('ping:', self)

class B(A):
    def pong(self):
        print('pong:', self)
        
class C(A):
    def pong(self):
        print('PONG:', self)
        
class D(B, C):
    def ping(self):
        A.ping(self)   # 绕过方法解析顺序，必须显式的传入 self 参数
        print('post-ping', self)
        
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)
        
d = D()

display(
    d.pong(), 
    C.pong(d),
    d.ping(),
    d.pingpong(),
    D.__mro__,   # 方法解析顺序，(Method Resolution Order, MRO)
)

pong: <__main__.D object at 0x0000023FEE9005F8>
PONG: <__main__.D object at 0x0000023FEE9005F8>
ping: <__main__.D object at 0x0000023FEE9005F8>
post-ping <__main__.D object at 0x0000023FEE9005F8>
ping: <__main__.D object at 0x0000023FEE9005F8>
post-ping <__main__.D object at 0x0000023FEE9005F8>
ping: <__main__.D object at 0x0000023FEE9005F8>
pong: <__main__.D object at 0x0000023FEE9005F8>
pong: <__main__.D object at 0x0000023FEE9005F8>
PONG: <__main__.D object at 0x0000023FEE9005F8>


None

None

None

None

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [20]:
# 查看类继承属性
import io
import numbers

print(f"bool \t\t {bool.__mro__}")
print(f"io \t\t {io.BytesIO.__mro__}")
print(f"numbers \t {numbers.Integral.__mro__}")

bool 		 (<class 'bool'>, <class 'int'>, <class 'object'>)
io 		 (<class '_io.BytesIO'>, <class '_io._BufferedIOBase'>, <class '_io._IOBase'>, <class 'object'>)
numbers 	 (<class 'numbers.Integral'>, <class 'numbers.Rational'>, <class 'numbers.Real'>, <class 'numbers.Complex'>, <class 'numbers.Number'>, <class 'object'>)


In [ ]:
 v